<a href="https://colab.research.google.com/github/sudharshanpaul/Deep-Learning/blob/main/ANN/Keras_Tuner/Hidden_Layers%26Hidden_Neurons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [161]:
df = pd.read_csv('Real_Combine.csv')

**Air Quality Index prediction**

In [162]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [163]:
df.isnull().sum()

,0
T,0
TM,0
Tm,0
SLP,0
H,0
VV,0
V,0
VM,0
PM 2.5,1


In [164]:
df['PM 2.5'] = df['PM 2.5'].fillna(df['PM 2.5'].mean())

In [165]:
df.isnull().sum()

,0
T,0
TM,0
Tm,0
SLP,0
H,0
VV,0
V,0
VM,0
PM 2.5,0


In [166]:
X = df.iloc[:,:-1]  # Independent features
y = df.iloc[:,-1]   # Dependent Feature

#### Hyper Parameters
- How many number of hidden layers we should have?
- How many number of Neurons we should have in hidden layers?
- Learning Rate

In [167]:
def build_model(hp):
  model = keras.Sequential()
  for i in range(hp.Int('num_layers', 2, 20)):
    model.add(layers.Dense(
            units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
            activation='relu'
        ))
  model.add(layers.Dense(1, activation='linear'))
  model.compile(
      optimizer=keras.optimizers.Adam(
          hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])
      ),
      loss='mean_absolute_error',
      metrics=['mean_absolute_error']
  )

  return model

In [168]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index'
)

In [169]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05], 'ordered': True}


In [170]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [171]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [172]:
tuner.search(X_train, y_train, epochs=5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 21s]
val_mean_absolute_error: 68.16650009155273

Best val_mean_absolute_error So Far: 38.056785583496094
Total elapsed time: 00h 02m 26s


In [173]:
tuner.results_summary()

Results summary
Results in project/Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 2 summary
Hyperparameters:
num_layers: 12
units_0: 352
units_1: 416
learning_rate: 0.001
units_2: 288
units_3: 256
units_4: 96
units_5: 416
units_6: 384
units_7: 256
units_8: 96
units_9: 32
units_10: 32
units_11: 32
Score: 38.056785583496094

Trial 3 summary
Hyperparameters:
num_layers: 9
units_0: 224
units_1: 384
learning_rate: 0.001
units_2: 224
units_3: 96
units_4: 512
units_5: 512
units_6: 32
units_7: 160
units_8: 320
units_9: 64
units_10: 256
units_11: 352
Score: 38.08705393473307

Trial 4 summary
Hyperparameters:
num_layers: 4
units_0: 288
units_1: 384
learning_rate: 0.0001
units_2: 128
units_3: 192
units_4: 480
units_5: 352
units_6: 448
units_7: 192
units_8: 352
units_9: 256
units_10: 256
units_11: 256
Score: 68.16650009155273

Trial 1 summary
Hyperparameters:
num_layers: 7
units_0: 192
units_1: 224
learning_rate: 1e-05
units_2: 160
units_